In [19]:
import pandas as pd
import numpy as np
import random
import umap
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'umap'

In [20]:
df = pd.read_csv("matching2.csv")
df.head()

,jobID,userID,like,description,location,jobTitle,hashtags
0,5f3ba0d0809ba90012487452,5f2c43b515633f0012d6db66,False,Für deine Arbeit als Bauzeichner ist Gewissenh...,Karlsruhe,Schülerpraktikum Bauzeichner,"Parktikum, Schüler, Bauzeichner, Bau, Zeichnen"
1,5f3ba0ae809ba9001248744e,5f2c43b515633f0012d6db66,False,-Eigenständige Durchführung von Instandhaltung...,Mannheim,Mechaniker (m/w/d),"Trenkwalder, Mannheim, Baustoffe, Mechaniker"
2,5f3b9ee1809ba90012487449,5f2c43b515633f0012d6db66,False,NaN,NaN,NaN,NaN
3,5f3b9d1b809ba9001248743e,5f2c43b515633f0012d6db66,False,Während deiner Ausbildung lernst du den sicher...,Karlsruhe,Ausbildung zum Augenoptiker (m/w/d),"Optiker, Ausbildung, Augen, Sehen"
4,5f3b7a10809ba9001248740c,5f2c43b515633f0012d6db66,False,Meisterliche „Ausbau-Fähigkeiten“ verdienen me...,Mannheim,Schreinermeister (m/w/d),"WERU, Martin, Mannheim, Festanstellung, Unbefr..."


In [21]:
df.shape

(4904, 7)

In [22]:
df.isnull().sum()

jobID           0
userID          0
like            0
description    52
location       52
jobTitle       52
hashtags       52
dtype: int64

In [23]:
# remove missing values
df.dropna(inplace=True)

In [24]:
df.shape

(4852, 7)

In [25]:
users = df["userID"].unique().tolist()
len(users)

67

In [26]:
# shuffle user ID's
random.shuffle(users)

# extract 90% of user ID's
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['userID'].isin(users_train)]
validation_df = df[~df['userID'].isin(users_train)]

In [27]:
# swipe history training
likes_train = []

# add liked jobs to list
for i in tqdm(users_train):
    temp = train_df[train_df["userID"] == i]["jobID"].tolist()
    likes_train.append(temp)

100%|██████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1395.60it/s]


In [28]:
# swipe history testing
likes_val = []

# add liked jobs to list
for i in tqdm(validation_df["userID"].unique()):
    temp = validation_df[validation_df["userID"] == i]["jobID"].tolist()
    likes_val.append(temp)

100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1744.62it/s]


In [59]:
model = Word2Vec(window = 2, sg = 0, hs = 1,
                negative = 10,
                alpha = 0.03, min_alpha=0.0007,
                seed = 13)

model.build_vocab(likes_train, progress_per=200)

model.train(likes_train, total_examples = model.corpus_count,
            epochs=10, report_delay=1)

(20126, 43480)

In [60]:
print(model)

Word2Vec(vocab=119, size=100, alpha=0.03)


In [61]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(119, 100)

In [62]:
jobs = train_df[["jobID","jobTitle"]]

# remove duplicates
jobs.drop_duplicates(inplace=True, subset='jobID', keep="last")

job_dict = jobs.groupby("jobID")["jobTitle"].apply(list).to_dict()

In [63]:
job_dict["5f3ba0d0809ba90012487452"]

['Schülerpraktikum Bauzeichner']

In [76]:
def similar_jobs(v, n = 6):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    print(ms)
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (job_dict[j[0]][0], j[1])
        #pair = (j[0], j[1]) use this to get back jobIds with similarity
        new_ms.append(pair)
        
    return new_ms

In [77]:
similar_jobs(model["5f3ba0d0809ba90012487452"])

[('5f3ba0ae809ba9001248744e', 0.992262601852417), ('5f46c4020b4ceb0012577514', 0.985022783279419), ('5f3b7a10809ba9001248740c', 0.9767637252807617), ('5f3b9d1b809ba9001248743e', 0.9755334854125977), ('5f46c88b0b4ceb001257751e', 0.9721207022666931), ('5f3b78a4809ba900124873fa', 0.9643678665161133)]


[('5f3ba0ae809ba9001248744e', 0.992262601852417),
 ('5f46c4020b4ceb0012577514', 0.985022783279419),
 ('5f3b7a10809ba9001248740c', 0.9767637252807617),
 ('5f3b9d1b809ba9001248743e', 0.9755334854125977),
 ('5f46c88b0b4ceb001257751e', 0.9721207022666931),
 ('5f3b78a4809ba900124873fa', 0.9643678665161133)]